In [1]:
import sys

sys.path.append('..')

from clingo import Control
from action_language_interpreter.util import solve
from action_language_interpreter.action_language import from_clingo_symbols, FluentLiteral

In [2]:
robot = """

causes(move, battery(B-1), battery(B)) :- battery(B), battery(B-1).
causes(move, position(P+1), position(P)) :- position(P), position(P+1).

position(1..3).
battery(0..1).

"""

answer_sets = solve(robot)

Answer  1: { battery(0) battery(1) position(1) position(2) position(3) causes(move,battery(0),battery(1)) causes(move,position(2),position(1)) causes(move,position(3),position(2)) }.
SAT 1 


In [3]:
answer_sets

[{Function('battery', [Number(0)], True),
  Function('battery', [Number(1)], True),
  Function('position', [Number(1)], True),
  Function('position', [Number(2)], True),
  Function('position', [Number(3)], True),
  Function('causes', [Function('move', [], True), Function('battery', [Number(0)], True), Function('battery', [Number(1)], True)], True),
  Function('causes', [Function('move', [], True), Function('position', [Number(2)], True), Function('position', [Number(1)], True)], True),
  Function('causes', [Function('move', [], True), Function('position', [Number(3)], True), Function('position', [Number(2)], True)], True)}]

In [4]:
AD = from_clingo_symbols(answer_sets[0])

In [5]:
AD

({move causes battery(0) if battery(1), move causes position(3) if position(2), move causes position(2) if position(1)}, set(), set())

In [6]:
AD.execute({FluentLiteral("battery(1)"), FluentLiteral("position(1)")}, 'move')

({battery(0), battery(1), position(1), position(2)},
 [move causes battery(0) if battery(1),
  move causes position(2) if position(1)])

In [7]:
instance = """

causes(move,position(2),position(1)).
causes(move,position(3),position(2)).
causes(move,battery(0),battery(1)).

"""

In [8]:
trace = """

holds_at(position(1), 1).
holds_at(battery(1), 1).
occurs_at(move, 1).
holds_at(position(2), 2).
holds_at(battery(0), 2).

outcome(position(2)).

"""

In [9]:
explanation_first = """

outcome(position(2)).

event_directEffects_at(move, position(2), 1).
event_directEffects_at(move, battery(0), 1).

# indirect effects empty

firstCausalExplanationDirectEffects(Effect) :- event_directEffects_at(Event, Effect, T), outcome(Effect).

"""

In [10]:
explanation_second = """

remainingOutcomeLiterals(Lit) :- not outcome(Lit), holds_at(Lit, O).

"""

In [11]:
explanation_third = """


"""